In [7]:
!pip install pyspark

In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, PCA
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import month, col, to_date
from pyspark.sql.functions import col, when, isnull
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit, year, current_date
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import avg
from pyspark.sql.functions import date_format



In [7]:
# 1. Criar sessão Spark
spark = SparkSession.builder.getOrCreate()

In [3]:
# 2. Carregar CSV
df_evasao = spark.read.option('header', 'true').csv('evasao_sintetica.csv')
df_evasao.show(10)

+----------+---------+-----------+-----+--------------+----+-----------+---+------+--------------------+--------------------+----------------+---------------+---------------+---------+-------------+--------------+-----------+---------------------+--------------------+-----------+------------+
|student_id|school_id|school_name|state|  municipality|year|grade_level|age|gender|socio_economic_class|    parent_education|family_income_R$|previous_repeat|attendance_rate|avg_score|special_needs|household_size|urban_rural|teacher_student_ratio|infrastructure_score|dropped_out|dropout_date|
+----------+---------+-----------+-----+--------------+----+-----------+---+------+--------------------+--------------------+----------------+---------------+---------------+---------+-------------+--------------+-----------+---------------------+--------------------+-----------+------------+
|  ST000001|    S0198| Escola 198|   RJ|       Niterói|2021|  9º Ano EF| 14|     F|               Baixa|            Su

In [12]:
# 3. Excluir valores nulos
df_limpo = df_evasao.dropna()


In [13]:
# 4. Formatar data
df_evasao = df_evasao.withColumn(
    "data_do_abandono_formatada",
    date_format("dropout_date", "dd/MM/yyyy")
)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `dropout_date` cannot be resolved. Did you mean one of the following? [`Escola_id`, `Estado`, `Idade`, `Ano`, `Urbano_Rural`].;
'Project [Estudante_id#314, Escola_id#338, Nome_da_escola#362, Estado#386, Municipio#434, Ano#410, Nivel_escolar#458, Idade#482, Genero#506, Classe_socio_economica#530, Educação_dos_pais#554, Renda_familiar#578, Repetiu_anteriormente#602, Taxa_de_comparecimento#626, Pontuação_média#650, Necessidades_especiais#674, Tamanho_da _família#698, Urbano_Rural#722, Proporção_professor_aluno#746, Pontuação_de_infraestrutura#770, Abandono#794, data_de_abandono#818, date_format('dropout_date, dd/MM/yyyy, Some(Etc/UTC)) AS data_do_abandono_formatada#980]
+- Project [Estudante_id#314, Escola_id#338, Nome_da_escola#362, Estado#386, Municipio#434, Ano#410, Nivel_escolar#458, Idade#482, Genero#506, Classe_socio_economica#530, Educação_dos_pais#554, Renda_familiar#578, Repetiu_anteri

In [11]:
# 5. Renomear colunas
rename_dict = {
    "student_id": "Estudante_id",
    "school_id": "Escola_id",
    "school_name": "Nome_da_escola",
    "state": "Estado",
    "year": "Ano",
    "municipality": "Municipio",
    "grade_level": "Nivel_escolar",
    "age": "Idade",
    "gender": "Genero",
    "socio_economic_class": "Classe_socio_economica",
    "parent_education": "Educação_dos_pais",
    "family_income_R$": "Renda_familiar",
    "previous_repeat": "Repetiu_anteriormente",
    "attendance_rate": "Taxa_de_comparecimento",
    "avg_score": "Pontuação_média",
    "special_needs": "Necessidades_especiais",
    "household_size": "Tamanho_da _família",
    "urban_rural": "Urbano_Rural",
    "teacher_student_ratio": "Proporção_professor_aluno",
    "infrastructure_score": "Pontuação_de_infraestrutura",
    "dropped_out": "Abandono",
    "dropout_date": "data_de_abandono"


}

for old_col, new_col in rename_dict.items():
    df_evasao = df_evasao.withColumnRenamed(old_col, new_col)

df_evasao.show(20)

+------------+---------+--------------+------+-----------------+----+-------------+-----+------+----------------------+--------------------+--------------+---------------------+----------------------+---------------+----------------------+-------------------+------------+-------------------------+---------------------------+--------+----------------+--------------------------+
|Estudante_id|Escola_id|Nome_da_escola|Estado|        Municipio| Ano|Nivel_escolar|Idade|Genero|Classe_socio_economica|   Educação_dos_pais|Renda_familiar|Repetiu_anteriormente|Taxa_de_comparecimento|Pontuação_média|Necessidades_especiais|Tamanho_da _família|Urbano_Rural|Proporção_professor_aluno|Pontuação_de_infraestrutura|Abandono|data_de_abandono|data_do_abandono_formatada|
+------------+---------+--------------+------+-----------------+----+-------------+-----+------+----------------------+--------------------+--------------+---------------------+----------------------+---------------+----------------------+-

In [15]:
# 6.Remover coluna
df_evasao = df_evasao.drop("data_de_abandono")



+------------+---------+--------------+------+-----------------+----+-------------+-----+------+----------------------+--------------------+--------------+---------------------+----------------------+---------------+----------------------+-------------------+------------+-------------------------+---------------------------+--------+--------------------------+
|Estudante_id|Escola_id|Nome_da_escola|Estado|        Municipio| Ano|Nivel_escolar|Idade|Genero|Classe_socio_economica|   Educação_dos_pais|Renda_familiar|Repetiu_anteriormente|Taxa_de_comparecimento|Pontuação_média|Necessidades_especiais|Tamanho_da _família|Urbano_Rural|Proporção_professor_aluno|Pontuação_de_infraestrutura|Abandono|data_do_abandono_formatada|
+------------+---------+--------------+------+-----------------+----+-------------+-----+------+----------------------+--------------------+--------------+---------------------+----------------------+---------------+----------------------+-------------------+------------+--

In [18]:
# 7. Taxa de evasão geral
taxa_evasao = df_evasao.groupBy("dropped_out") \
                .count() \
                .withColumnRenamed("count", "total") \
                .withColumn("percentual", (col("total") / total_registros) * 100)
taxa_evasao.show()



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `dropped_out` cannot be resolved. Did you mean one of the following? [`Estado`, `Genero`, `Abandono`, `Ano`, `Escola_id`].;
'Aggregate ['dropped_out], ['dropped_out, count(1) AS count#1413L]
+- Project [Estudante_id#314, Escola_id#338, Nome_da_escola#362, Estado#386, Municipio#434, Ano#410, Nivel_escolar#458, Idade#482, Genero#506, Classe_socio_economica#530, Educação_dos_pais#554, Renda_familiar#578, Repetiu_anteriormente#602, Taxa_de_comparecimento#626, Pontuação_média#650, Necessidades_especiais#674, Tamanho_da _família#698, Urbano_Rural#722, Proporção_professor_aluno#746, Pontuação_de_infraestrutura#770, Abandono#794, data_do_abandono_formatada#175 AS data_do_abandono#1229]
   +- Project [Estudante_id#314, Escola_id#338, Nome_da_escola#362, Estado#386, Municipio#434, Ano#410, Nivel_escolar#458, Idade#482, Genero#506, Classe_socio_economica#530, Educação_dos_pais#554, Renda_familiar#578, 

In [19]:
# 8. Média de presença por classe socioeconômica
df_evasao.groupBy("socio_economic_class") \
  .agg(avg("attendance_rate").alias("media_frequencia")) \
  .show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `socio_economic_class` cannot be resolved. Did you mean one of the following? [`Classe_socio_economica`, `Nivel_escolar`, `Nome_da_escola`, `Abandono`, `Educação_dos_pais`].;
'Aggregate ['socio_economic_class], ['socio_economic_class, avg('attendance_rate) AS media_frequencia#1437]
+- Project [Estudante_id#314, Escola_id#338, Nome_da_escola#362, Estado#386, Municipio#434, Ano#410, Nivel_escolar#458, Idade#482, Genero#506, Classe_socio_economica#530, Educação_dos_pais#554, Renda_familiar#578, Repetiu_anteriormente#602, Taxa_de_comparecimento#626, Pontuação_média#650, Necessidades_especiais#674, Tamanho_da _família#698, Urbano_Rural#722, Proporção_professor_aluno#746, Pontuação_de_infraestrutura#770, Abandono#794, data_do_abandono_formatada#175 AS data_do_abandono#1229]
   +- Project [Estudante_id#314, Escola_id#338, Nome_da_escola#362, Estado#386, Municipio#434, Ano#410, Nivel_escolar#458, Id

In [20]:
# 9. Filtrar apenas alunos que evadiram
evasores = df_evasao.filter(col("dropped_out") == 1)
evasores.show(5)

+------------+---------+--------------+------+---------+----+-------------+-----+------+----------------------+--------------------+--------------+---------------------+----------------------+---------------+----------------------+-------------------+------------+-------------------------+---------------------------+--------+----------------+
|Estudante_id|Escola_id|Nome_da_escola|Estado|Municipio| Ano|Nivel_escolar|Idade|Genero|Classe_socio_economica|   Educação_dos_pais|Renda_familiar|Repetiu_anteriormente|Taxa_de_comparecimento|Pontuação_média|Necessidades_especiais|Tamanho_da _família|Urbano_Rural|Proporção_professor_aluno|Pontuação_de_infraestrutura|Abandono|data_do_abandono|
+------------+---------+--------------+------+---------+----+-------------+-----+------+----------------------+--------------------+--------------+---------------------+----------------------+---------------+----------------------+-------------------+------------+-------------------------+--------------------

In [21]:
# 10. Salvar resultado filtrado (opcional)
evasores.write.mode("overwrite").csv("evasores_tratados.csv", header=True)

In [59]:
# 11. Encerrar sessão
spark.stop()